<a href="https://colab.research.google.com/github/JordanFoss/STAT3007_Project/blob/main/Conv_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/JordanFoss/STAT3007_Project.git
%cd STAT3007_Project/

Cloning into 'STAT3007_Project'...
remote: Enumerating objects: 3979, done.
remote: Counting objects: 100% (257/257), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 3979 (delta 112), reused 169 (delta 50), pack-reused 3722
Receiving objects: 100% (3979/3979), 677.11 MiB | 25.26 MiB/s, done.
Resolving deltas: 100% (697/697), done.
Checking out files: 100% (2854/2854), done.
/content/STAT3007_Project


In [14]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor
import scipy
import numpy as np
import librosa
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.autograd import Variable
import pandas as pd
import glob
import os
from librosa import display
from torchsummary import summary

from IPython.display import Audio

In [4]:
!pip install colorednoise as cn

  Created wheel for colorednoise: filename=colorednoise-1.1.1-cp37-none-any.whl size=3958 sha256=efa52ddb673e3073dfbf929692307fa0806a3b6e687866f5905fa56a9a169abd
  Stored in directory: /root/.cache/pip/wheels/84/be/f3/3e7e1c80ebab3f6f0dbd3e34e787b902d2280d66706485fef4
Successfully built colorednoise


In [49]:
import pre_process
import data_loading
import CNN_Model
from data_loading import *

In [50]:
X,y = data_loading.load_samples(os.getcwd())

In [134]:
X = torch.tensor(X)
X = X.reshape(X.shape[0],1,X.shape[1],X.shape[2])
y = torch.tensor(y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


In [135]:
X.shape

torch.Size([480, 1, 128, 63])

In [136]:
cnn = CNN_Model.ConvNet()

In [137]:
summary(cnn, (1,128,21))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 14, 127, 19]              98
              ReLU-2          [-1, 14, 127, 19]               0
         MaxPool2d-3            [-1, 14, 64, 9]               0
           Dropout-4            [-1, 14, 64, 9]               0
            Conv2d-5            [-1, 24, 63, 7]           2,040
              ReLU-6            [-1, 24, 63, 7]               0
         MaxPool2d-7            [-1, 24, 32, 3]               0
Total params: 2,138
Trainable params: 2,138
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.82
Params size (MB): 0.01
Estimated Total Size (MB): 0.84
----------------------------------------------------------------


In [138]:
# RNN combined with CNN
class LRCN(nn.Module):
    def __init__(self, CNN, shape = (24,32,3)):
        super(LRCN, self).__init__()

        self.cnn = CNN
        self.shape = shape

        channel, freq, times = shape
        self.lstm_layers = nn.LSTM(freq*channel*times,256,num_layers = 2, bidirectional = True)
        self.linear = nn.Sequential(nn.Linear(256*2, 5))
        self.flatten = nn.Flatten()

    def forward(self, x, step_size = 21, use_cuda = False):

      if use_cuda:
        h_t = torch.zeros(4,x.shape[0] ,256, dtype=torch.float).to(x.device)
        c_t = torch.zeros(4,x.shape[0], 256, dtype=torch.float).to(x.device)

      else:
        h_t = torch.zeros(4,x.shape[0], 256, dtype=torch.float)
        c_t = torch.zeros(4,x.shape[0], 256, dtype=torch.float)
      
      look_ahead_time = 21
      for current_time in range(0,x.shape[-1], step_size):

        x_t = x[:,:,:,current_time:current_time+look_ahead_time]
        conv_x = self.cnn(x_t)

        conv_x_flat =  self.flatten(conv_x)

        conv_x_flat = conv_x_flat.reshape(1,conv_x_flat.shape[0],conv_x_flat.shape[1])

        output, (h_t, c_t) = self.lstm_layers(conv_x_flat, (h_t, c_t))

      decision_vec = self.linear(output[0])
      return decision_vec

In [139]:
conv_LSTM = LRCN(cnn)

In [140]:
conv_LSTM(X[:30]).shape

torch.Size([30, 5])

In [141]:
data_sets = data_loading.load_sets(X,y,train_ratio=[0.8], seed = [10])

In [142]:
data_train, data_test = data_sets[0]

In [143]:
def train_model(data_train, data_test, net, loss, nepoch ,lr = 0.01, batch_size = -1, use_cuda = False, print_output = True):

  # appropriate data type for CPU or GPU
  device = None
  if use_cuda and torch.cuda.is_available():
    dtype = torch.cuda.FloatTensor
    device = torch.device("cuda")
    net = net.to(device)
  else:
    dtype = torch.FloatTensor

  optimizer = optim.SGD(net.parameters(), lr = lr)
  data_train = data_train.change_type(dtype)
  data_test = data_test.change_type(dtype)

  data_loader = DataLoader(data_train, batch_size = batch_size, shuffle = True)

  for epoch in range(nepoch):
    for X_batch, y_batch in data_loader:
      y_batch = y_batch.type(torch.LongTensor)
      if use_cuda and device != None:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        y_batch = y_batch.type(torch.cuda.LongTensor)

      optimizer.zero_grad()

      # since all our values are negative, we convert them to positive


      pred = net(X_batch, use_cuda = use_cuda)
      Rn = loss(pred, y_batch)
      Rn.backward()
      optimizer.step()

    if print_output:
      print('epoch:', epoch)
      print('loss:',Rn.item())
      print('------------')

  print('final loss:', Rn.item())

  return net

In [144]:
loss = nn.CrossEntropyLoss()

In [148]:
nepoch = 10
trained_net = train_model(data_train, data_test, conv_LSTM, loss, nepoch=nepoch , batch_size = 10,lr = 0.01, use_cuda = True, print_output = True)

epoch: 0
loss: 1.6191939115524292
------------
epoch: 1
loss: 1.5839289426803589
------------
epoch: 2
loss: 1.5451276302337646
------------
epoch: 3
loss: 1.3351891040802002
------------
epoch: 4
loss: 1.4917514324188232
------------
epoch: 5
loss: 1.3749934434890747
------------
epoch: 6
loss: 1.163201093673706
------------
epoch: 7
loss: 1.2972311973571777
------------
epoch: 8
loss: 1.4223605394363403
------------
epoch: 9
loss: 1.1163952350616455
------------
final loss: 1.1163952350616455
